In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings
import optuna

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [3]:
file_path = 'processed_datasetNew.csv'
data = pd.read_csv(file_path)

data = data.set_index('Date')
data.index = pd.to_datetime(data.index)

In [4]:
split_date = pd.to_datetime('2008-12-31 23:59')
data = data[data.index >= split_date]

In [5]:
data["Temperature t+24"] = data["Temperature"].shift(periods=-24)
data["Humidity t+24"] = data["Humidity"].shift(periods=-24)
data["Month"] = data.index.month
data["Month_sin"] = np.sin(2 * np.pi * data.index.month / 12)
data["Month_cos"] = np.cos(2 * np.pi * data.index.month / 12)
data["Month t+24"] = data["Month"].shift(periods=-24)
data["Month_sin t+24"] = data["Month_sin"].shift(periods=-24)
data["Month_cos t+24"] = data["Month_cos"].shift(periods=-24)
data["Quarter"] = data.index.quarter
data["Quarter_sin"] = np.sin(2 * np.pi * data.index.quarter / 4)
data["Quarter_cos"] = np.cos(2 * np.pi * data.index.quarter / 4)
data["Quarter t+24"] = data["Quarter"].shift(periods=-24)
data["Quarter_sin t+24"] = data["Quarter_sin"].shift(periods=-24)
data["Quarter_cos t+24"] = data["Quarter_cos"].shift(periods=-24)
data["Year"] = data.index.year
data["Year t+24"] = data["Year"].shift(periods=-24)
data['week_cos'] =  np.cos(2 * np.pi * data.index.isocalendar().week / 53)
data['week_sin'] =  np.sin(2 * np.pi * data.index.isocalendar().week / 53)
data["week_sin t+24"] = data["week_sin"].shift(periods=-24)
data["week_cos t+24"] = data["week_cos"].shift(periods=-24)
data['weekday_cos'] = np.sin(2 * np.pi * (data.index.weekday+1) / 7)
data['weekday_sin'] = np.cos(2 * np.pi * (data.index.weekday+1) / 7)
data["weekday_sin t+24"] = data["weekday_sin"].shift(periods=-24)
data["weekday_cos t+24"] = data["weekday_cos"].shift(periods=-24)

In [6]:
features_options = [
    ["Temperature", "Humidity"],
    ["Temperature", "Humidity", "Year", "Month"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "weekday_cos", "weekday_sin"],
    ["Temperature", "Humidity", "Temperature t+24", "Humidity t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Temperature t+24", "Humidity t+24", "Year t+24", "Month t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "Temperature t+24", "Humidity t+24", "Year t+24", 
    "Month t+24", "Month_cos t+24", "Month_sin t+24"],
    ["Temperature", "Humidity", "Year", "Month", "Month_cos", "Month_sin", "weekday_cos", "weekday_cos", "Temperature t+24", "Humidity t+24",
    "Year t+24", "Month t+24", "Month_cos t+24", "Month_sin t+24", "weekday_cos t+24", "weekday_sin t+24"],
    ["Temperature", "Humidity", "Year", "weekday_cos", "weekday_sin", "Month_cos", "Month_sin"],
    ["Temperature", "Humidity", "Year", "weekday_cos", "weekday_sin", "Temperature t+24", "Humidity t+24", "Year t+24", "weekday_cos t+24", "weekday_sin t+24"]
]

In [7]:
data_sin = data[['SIN']]
data_cov = data[features_options[8]]

In [8]:
series = TimeSeries.from_dataframe(data_sin, value_cols=["SIN"])
covariates = TimeSeries.from_dataframe(data_cov, value_cols=features_options[8])
series = series.astype(np.float32)
covariates = covariates.astype(np.float32)

In [9]:
# Add holiday binary value
covariates = covariates.add_holidays('PY')

In [10]:
# # Create training and validation sets:
training_cutoff = pd.Timestamp("20191231T230000")
train, val = series.split_after(training_cutoff)
# validation_cutoff = pd.Timestamp("20211231T230000")
# val, test = val.split_after(validation_cutoff)

In [11]:
# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [12]:
covariates = covariates.stack(
    TimeSeries.from_times_and_values(
        times=series.time_index,
        values=np.arange(len(series)),
        columns=["linear_increase"],
    )
)
covariates = covariates.astype(np.float32)
cov_train, cov_val = covariates.split_after(training_cutoff)
# cov_val, cov_test = covariates.split_after(validation_cutoff)

In [13]:
# transform covariates (note: we fit the transformer on train split and can then transform the entire covariates series)
scaler_covs = Scaler()
covariates_transformed_train = scaler_covs.fit_transform(cov_train)
covariates_transformed_val = scaler_covs.transform(cov_val)
covariates_transformed = scaler_covs.transform(covariates)

In [14]:
from darts.models import TFTModel, ARIMA, RNNModel
from pytorch_lightning.callbacks import EarlyStopping
from torchmetrics import MeanAbsolutePercentageError

torch_metrics = MeanAbsolutePercentageError()
torch.manual_seed(42)

# early stop callback
my_stopper = EarlyStopping(
    monitor="val_MeanAbsolutePercentageError", # "train_loss"
    patience=2,
    min_delta=0.0001,
    mode='min',
)
pl_trainer_kwargs={
      "accelerator": "gpu",
      "devices": [0],
      "callbacks": [my_stopper]
}

tft = TFTModel(
    input_chunk_length=20*24,
    output_chunk_length=1*24,
    hidden_size=50,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.2680939780722607,
    batch_size=21,
    n_epochs=7,
    add_relative_index=False,
    add_encoders=None,
    # loss_fn=MSELoss(),
    random_state=42,
    force_reset = True,
    save_checkpoints=True,
    torch_metrics=torch_metrics,
    model_name="tft_best_model_19",
    pl_trainer_kwargs=pl_trainer_kwargs,
)
tft.fit(series=train_transformed, val_series=val_transformed, future_covariates=covariates_transformed, val_future_covariates=covariates_transformed_val, num_loader_workers=2, verbose=True)
tft = TFTModel.load_from_checkpoint("tft_best_model_19")

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [15]:
# Save the trained TFT model
tft_model_path = "tft_best_model_19.pt"
tft.save(tft_model_path)

In [16]:
n = len(val)  # Get the length of the validation dataset
tft_pred_val = tft.predict(n=n, future_covariates=covariates_transformed)
tft_pred_val = transformer.inverse_transform(tft_pred_val)

Predicting: 0it [00:00, ?it/s]

In [17]:
validation_range = ('2020-01-01', '2022-12-31')
dtin = pd.date_range(start=validation_range[0], end=validation_range[1], freq='H')

In [18]:
# Convert TimeSeries to numpy arrays
tft_pred_val_arr = tft_pred_val.values()
val_arr = val.values()

In [19]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error,max_error

ixmax=np.argmax(val_arr.reshape(-1,24),axis=1)
hmaxerr=((tft_pred_val_arr.reshape(-1,24)[np.arange(tft_pred_val_arr.reshape(-1,24).shape[0]),ixmax]-np.max(val_arr.reshape(-1,24),axis=1))/np.max(val_arr.reshape(-1,24),axis=1))
hsup=np.percentile(hmaxerr,99)
hinf=np.percentile(hmaxerr,1)
mse=mean_squared_error( val_arr, tft_pred_val_arr)
rmse=np.sqrt(mse)
error=np.abs(val_arr-tft_pred_val_arr)
Error_95_=np.percentile(error,95)
maxError_=max_error(val_arr, tft_pred_val_arr)

r2_=r2_score(val_arr, tft_pred_val_arr)
print( "MSE: " + str(mse)+" RMSE:" + str(rmse  ))
print( "MaxError: " + str(maxError_)+" R2:" + str(r2_  )+" Error95:" + str(Error_95_))
print( "hsup: " + str(hsup)+" hinf:" + str(hinf))
print( "hmaxerr5: " + str(np.percentile(error,5)))
print( "hmaxerr50: " + str(np.percentile(error,50)))
print( "hmaxerr95: " + str(np.percentile(error,95)))

MSE: 92928.09 RMSE:304.8411
MaxError: 1700.0289 R2:0.679322645459794 Error95:641.9087890625
hsup: 0.17474301084876015 hinf:-0.3275212347507477
hmaxerr5: 14.579534912109379
hmaxerr50: 166.764404296875
hmaxerr95: 641.9087890625
